In [1]:
import os
import numpy as np
import pandas as pd
import nltk.tokenize
import re
import random
from nltk.util import ngrams
import tqdm
from nltk.tokenize import RegexpTokenizer
import torch

In [2]:
from transformers import LEDTokenizer, LongformerTokenizer, LEDForConditionalGeneration
import torch
from transformers import TrainingArguments, Trainer

### Option: Initialize Model

In [ ]:
generate_model = LEDForConditionalGeneration.from_pretrained("allenai/longformer-base-4096", 
                max_encoder_position_embeddings = 2048, max_position_embeddings = 1536) 
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

In [ ]:
generate_model.config

In [ ]:
tokenizer.pad_token_id

### Option: Load Model

In [3]:
model_dir = "./output17/checkpoint-9000/"

generate_model = LEDForConditionalGeneration.from_pretrained(model_dir) 
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.input_ids = encodings.input_ids
        self.attention_mask = encodings.attention_mask
        result = []
        for label in labels.input_ids:
            tmp =list(label)
            result.append([-100 if token_id == tokenizer.pad_token_id else token_id for token_id in label])
        self.labels = result

    def __getitem__(self, idx):
        item = {}
        item["input_ids"] = self.input_ids[idx]
        item["attention_mask"] = self.attention_mask[idx]
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.input_ids)

In [5]:
train_dataset = torch.load("train_dataset_b.pth")
test_dataset = torch.load("test_dataset.pth")

In [6]:
args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=2,
    save_steps = 6750
)
trainer = Trainer(
    model=generate_model,
    args=args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [ ]:
trainer.train()

/home/guanghax/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 47722
  Num Epochs = 2
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 3
  Gradient Accumulation steps = 1
  Total optimization steps = 31816


Step,Training Loss
500,2.099400
1000,2.134100
1500,2.045800
2000,2.069200
2500,2.051400
3000,2.030900
3500,2.028300
4000,2.043900
4500,2.001200
5000,2.053500


Saving model checkpoint to output18/checkpoint-6750
Configuration saved in output18/checkpoint-6750/config.json
Model weights saved in output18/checkpoint-6750/pytorch_model.bin
Saving model checkpoint to output18/checkpoint-13500
Configuration saved in output18/checkpoint-13500/config.json
Model weights saved in output18/checkpoint-13500/pytorch_model.bin
